# 감성분석 모델 구축

In [ ]:
# ! pip install scikit-learn
# ! pip install konlpy

In [1]:
import re
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# from konlpy.tag import Mecab
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

warnings.filterwarnings(action = 'ignore') # warning 메시지 표시x

C:\Users\A\AppData\Local\Temp\ipykernel_16048\74047196.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### 1. 훈련모델Ⅰ - 200,000개 (7:3)
- 네이버 쇼핑 리뷰만 이용
- 7:3으로 나누고, 훈련 -> 평가 진행
- 15만개 - 테스트 / 5만개 - 평가

In [2]:
model1_only_naver = pd.read_excel('(0)naver_reviews.xlsx') # 200,000개

# 0. 훈련 데이터와 테스트 데이터를 3:1 비율로 분리
train_data, test_data = train_test_split(model1_only_naver, test_size = 0.25, random_state = 42)

print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))
# 훈련용 리뷰의 개수 : 150000
# 테스트용 리뷰의 개수 : 50000

훈련용 리뷰의 개수 : 150000
테스트용 리뷰의 개수 : 50000


In [3]:
# 1. 라벨 분포 확인

train_data['label'].value_counts()
# 0    75076
# 1    74924

test_data['label'].value_counts()
# 1    25039
# 0    24961

label
1    25039
0    24961
Name: count, dtype: int64

In [4]:
# 2. 한글 외의 문자 제거
    #  ‘ㄱ’으로 시작하거나 ‘가’부터 ‘힣’까지의 문자를 제외한 나머지는 공백으로 치환
    # ? 한계1) 이모티콘도 충분히 유효한 데이터이나 제거해버림..
    # ? 한계2) 영어도 제거..

train_data['reviews'] = train_data['reviews'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
test_data['reviews'] = test_data['reviews'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))

train_data.head()
test_data.head()

,ratings,reviews,label
119737,2,마감이 안좋아요 실밥도 많고 바느질도 부족한 부분이 몇군데 있네요 교환받기 귀찮아서...,0
72272,5,깨끗하게 잘 다듬어져 있어요 맛도좋고요,1
158154,4,재구매 배송빨라요 길냥이들이 잘먹어요 대용량이라 좋네요,1
65426,5,제품도 빨리 배송해주시고 꼼꼼하게 잘챙겨주셨어요,1
30074,5,기타 남 멋지고 예뻐요 여러 사은품도 좋아요,1


In [5]:
# 3. 분석 모델 구축 전 작업

# 1)형태소 분석 - 문장을 토큰화
okt = Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

# 2)TF-IDF 벡터화에 사용할 tfidf 객체 생성 -> 벡터로 변환(transform)
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(train_data['reviews'])
model1_train_tfidf = tfidf.transform(train_data['reviews'])
model1_train_tfidf
# 14분 17초

<150000x123858 sparse matrix of type '<class 'numpy.float64'>'
	with 3112293 stored elements in Compressed Sparse Row format>

In [6]:
# 4. 분석 모델 구축

SA_lr = LogisticRegression(random_state=0)

# 1)train_tfidf를 독립변수 X로 하고 label 컬럼을 종속 변수 Y로 하여, 로지스틱 회귀 모델SA_lr의 내부 설정값을 조정fit()
SA_lr.fit(model1_train_tfidf, train_data['label'])

# 2) 최적값(조정값)을 찾기 위해 그리드서치
    # 하이퍼 매개변수 C에 대해 비교 검사를 할 6개 값[1, 3, 3.5, 4, 4.5, 5]을 params로 하고,
    # 교차 검증cv을 3, 모형 비교 기준은 정확도로 설정(scoring='accuracy')하여 GridSearchCV 객체를 생성
params = {'C': [1,3,3.5,4,4.5,5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

# 3)GridSearchCV 객체에 nsmc_train_tfidf와 label 컬럼에 대해 설정값을 조정fit( )
SA_lr_grid_cv.fit(model1_train_tfidf, train_data['label'])

# 4)GridSearchCV에 의해 찾은 최적의 C 매개변수best_params와 최고 점수best_score를 출력하여 확인
print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

# 5)최적 매개변수가 설정된 모형best_estimator을 SA_lr_best 객체에 저장
    # 최적 매개변수의 best 모델 저장
SA_lr_best1 = SA_lr_grid_cv.best_estimator_ 


# Fitting 3 folds for each of 6 candidates, totalling 18 fits
# {'C': 4} 0.9128

Fitting 3 folds for each of 6 candidates, totalling 18 fits
{'C': 4} 0.9128


In [7]:
# 5. 분석 모델 평가 - 평가용 데이터를 이용하여 모델 정확도 확인하기

model1_test_tfidf = tfidf.transform(test_data['reviews'])

# "감성 분류 모델"인 SA_lr_best에 nsmc_test_tfidf 벡터를 사용하여 감성을 예측predict( )
test_predict = SA_lr_best1.predict(model1_test_tfidf)

# 평가용 데이터의 감성 결과값nsmc_test_df['label']과 감성 예측값test_predict을 기반으로 정확도를 계산accuracy_score( )하여출력

print('감성 분석 정확도 : ', round(accuracy_score(test_data['label'], test_predict), 3))
# 감성 분류 모델의 정확도가 91.6%!

감성 분석 정확도 :  0.916


In [8]:
# 6. 감성 분류 모델1 활용 -> 새 텍스트로 감성 예측 진행
st = input('감성 분석할 문장 입력>>')

# 1)입력 텍스트에 대한 전처리 수행
st = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(st) # ㄱ~, 가~힣만 포함시킨다는 뜻
st = [" ".join(st)]
print('0.입력문자:', st)

# 2)idf객체로 벡터화transform( ) 후에 모델에 적용하여 감성 예측predict( )을 수행
st_tfidf = tfidf.transform(st) # 1)입력 텍스트의 피처 벡터화
st_predict = SA_lr_best1.predict(st_tfidf) # 2)최적 감성 분석 모델에 적용하여 감성 분석 평가

# 3)예측값 출력하기
if(st_predict == 0):
    print('=========')
    print('1.결과:',st , "->> 부정 감성")
else :
    print('=========')
    print('1.결과:',st , "->> 긍정 감성")

0.입력문자: ['오오']
1.결과: ['오오'] ->> 긍정 감성


In [ ]:
# 시각화1
# train_data['label'].value_counts().plot(kind = 'bar')